# FINAL PROJECT VIX RAKAMIN HOME CREDIT INDONESIA
by: Ayu Rizki Herayati

A. DATASET YANG DIGUNAKAN
- Dari 7 sumber dataset yang telah diberikan, pada project ini hanya difokuskan pada salah satu sumber yakni 'application_train.csv'. Dataset ini berisi data statis seluruh aplikan/klien. Satu baris mewakili satu pinjaman dari data sampel. Pada dataset ini juga terdapat TARGET yang berisi tentang kesanggupan klien dalam menyelesaikan pinjaman.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
pd.options.mode.chained_assignment = None  # default='warn'
from imblearn.over_sampling import SMOTE

In [2]:
# Pengaturan agar dataset dapat ditampilkan keseluruhan
pd.set_option('display.max_columns', None)
pd.get_option('display.max_columns')

pd.set_option('display.max_rows', 200)
pd.get_option('display.max_rows')

200

In [3]:
# Membaca dataframe
app_train = pd.read_csv("application_train.csv")
# Menampilkan 5 data teratas dari dataset
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [4]:
# Menampilkan info statistik dataset (app_train)
print('Ukuran dataset:', app_train.shape)
print('\n Statistik deskriptif:\n', app_train.describe())
print('\n Tipe data dataset:\n', app_train.dtypes)
print('\n Dataset info:')
print(app_train.info())

Ukuran dataset: (307511, 122)

 Statistik deskriptif:
           SK_ID_CURR         TARGET   CNT_CHILDREN  AMT_INCOME_TOTAL  \
count  307511.000000  307511.000000  307511.000000      3.075110e+05   
mean   278180.518577       0.080729       0.417052      1.687979e+05   
std    102790.175348       0.272419       0.722121      2.371231e+05   
min    100002.000000       0.000000       0.000000      2.565000e+04   
25%    189145.500000       0.000000       0.000000      1.125000e+05   
50%    278202.000000       0.000000       0.000000      1.471500e+05   
75%    367142.500000       0.000000       1.000000      2.025000e+05   
max    456255.000000       1.000000      19.000000      1.170000e+08   

         AMT_CREDIT    AMT_ANNUITY  AMT_GOODS_PRICE  \
count  3.075110e+05  307499.000000     3.072330e+05   
mean   5.990260e+05   27108.573909     5.383962e+05   
std    4.024908e+05   14493.737315     3.694465e+05   
min    4.500000e+04    1615.500000     4.050000e+04   
25%    2.700000e+05  

In [5]:
# Mengecek nilai missing value dari dataset
print('Nilai missing value di dataset:\n')
print(app_train.isnull().sum())
print('Jumlah nilai missing value keseluruhan:', app_train.isnull().sum().sum())

Nilai missing value di dataset:

SK_ID_CURR                           0
TARGET                               0
NAME_CONTRACT_TYPE                   0
CODE_GENDER                          0
FLAG_OWN_CAR                         0
FLAG_OWN_REALTY                      0
CNT_CHILDREN                         0
AMT_INCOME_TOTAL                     0
AMT_CREDIT                           0
AMT_ANNUITY                         12
AMT_GOODS_PRICE                    278
NAME_TYPE_SUITE                   1292
NAME_INCOME_TYPE                     0
NAME_EDUCATION_TYPE                  0
NAME_FAMILY_STATUS                   0
NAME_HOUSING_TYPE                    0
REGION_POPULATION_RELATIVE           0
DAYS_BIRTH                           0
DAYS_EMPLOYED                        0
DAYS_REGISTRATION                    0
DAYS_ID_PUBLISH                      0
OWN_CAR_AGE                     202929
FLAG_MOBIL                           0
FLAG_EMP_PHONE                       0
FLAG_WORK_PHONE                

In [6]:
# Menyisipkan nilai median pada nilai missing value
app_train = app_train.fillna(app_train.median())

In [7]:
# Korelasi dataset
app_train_corr = app_train.corr()
print('Korelasi dataset:\n', app_train_corr)

Korelasi dataset:
                               SK_ID_CURR    TARGET  CNT_CHILDREN  \
SK_ID_CURR                      1.000000 -0.002108     -0.001129   
TARGET                         -0.002108  1.000000      0.019187   
CNT_CHILDREN                   -0.001129  0.019187      1.000000   
AMT_INCOME_TOTAL               -0.001820 -0.003982      0.012882   
AMT_CREDIT                     -0.000343 -0.030369      0.002145   
AMT_ANNUITY                    -0.000431 -0.012815      0.021377   
AMT_GOODS_PRICE                -0.000258 -0.039623     -0.001856   
REGION_POPULATION_RELATIVE      0.000849 -0.037227     -0.025573   
DAYS_BIRTH                     -0.001500  0.078239      0.330938   
DAYS_EMPLOYED                   0.001366 -0.044932     -0.239818   
DAYS_REGISTRATION              -0.000973  0.041975      0.183395   
DAYS_ID_PUBLISH                -0.000384  0.051457     -0.028019   
OWN_CAR_AGE                     0.001285  0.015982      0.026034   
FLAG_MOBIL                   

In [8]:
# Mengecek Distribusi TARGET
print('Distribusi Target:\n')
print(app_train.groupby(['TARGET'])['TARGET'].count())

# Plot Distribusi Target
plt.savefig('Target.png')
temp = app_train['TARGET'].value_counts()
df = {'data':[
    {'labels': temp.index,
    'values': temp.values,
    'type' : 'pie'}
]}
iplot(df)

Distribusi Target:

TARGET
0    282686
1     24825
Name: TARGET, dtype: int64


<Figure size 432x288 with 0 Axes>

Terlihat bahwa antara target 1 (klien yang kesulitan dalam pembayaran) jauh lebih sedikit dibandingkan target 0. Hal ini menyebabkan adanya imbalanced data, yakni suatu kondisi dimana distribusi antara kelas data tidak seimbang, jumlah kelas data yang satu lebih banyak dibanding kelas data yang lainnya. 

In [9]:
# Mengecek Tipe Pinjaman 
print('Tipe Pinjaman:\n')
print(app_train.groupby(['NAME_CONTRACT_TYPE'])['NAME_CONTRACT_TYPE'].count())

# Plot Distribusi Target
temp = app_train['NAME_CONTRACT_TYPE'].value_counts()
df = {'data':[
    {'labels': temp.index,
    'values': temp.values,
    'type' : 'pie'}
]}
iplot(df)

Tipe Pinjaman:

NAME_CONTRACT_TYPE
Cash loans         278232
Revolving loans     29279
Name: NAME_CONTRACT_TYPE, dtype: int64


Tipe pinjaman tunai (cash loans) lebih banyak

In [10]:
# Mengecek Distribusi Jenis Kelamin Klien
print('Distribusi Jenis Kelamin:\n')
print(app_train.groupby(['CODE_GENDER'])['CODE_GENDER'].count())

# Plot Jenis Kelamin
temp = app_train['CODE_GENDER'].value_counts()
df = {'data':[
    {'labels': temp.index,
     'values': temp.values,
     'type' : 'pie'
    }]
     }
iplot(df)

Distribusi Jenis Kelamin:

CODE_GENDER
F      202448
M      105059
XNA         4
Name: CODE_GENDER, dtype: int64


Terlihat dari gambar di atas bahwa klien yang berjenis kelamin perempuan lebih banyak mengajukan aplikasi dibanding klien berjenis kelamin laki-laki

In [11]:
# Mengecek Distribusi Tipe Pendapatan Klien
print('Distribusi Tipe Pendapatan:\n')
print(app_train.groupby(['NAME_INCOME_TYPE'])['NAME_INCOME_TYPE'].count())

# Plot Tipe Pendapatan
temp = app_train['NAME_INCOME_TYPE'].value_counts()
df = {'data':[
    {'labels': temp.index,
     'values': temp.values,
     'type' : 'pie'
    }]
     }
iplot(df)

Distribusi Tipe Pendapatan:

NAME_INCOME_TYPE
Businessman                 10
Commercial associate     71617
Maternity leave              5
Pensioner                55362
State servant            21703
Student                     18
Unemployed                  22
Working                 158774
Name: NAME_INCOME_TYPE, dtype: int64


Klien yang memiliki sumber pendapatan dengan bekerja merupakan tipe klien terbanyak diantara yang lainnya. Sementara itu klien dengan tipe sumber pendapatan dari bisnis, cuti hamil, siswa, atau tidak bekerja tidak muncul dalam plot karena jumlah klien yang sangat sedikit yakni kurang dari 50.

In [13]:
# Mengecek Distribusi Status Dalam Keluarga
print('Distribusi Status Klien Dalam Keluarga:\n')
print(app_train.groupby(['NAME_FAMILY_STATUS'])['NAME_FAMILY_STATUS'].count())

# Plot Distribusi Target
temp = app_train['NAME_FAMILY_STATUS'].value_counts()
df = {'data':[
    {'labels': temp.index,
    'values': temp.values,
    'type' : 'pie'}
]}
iplot(df)

Distribusi Status Klien Dalam Keluarga:

NAME_FAMILY_STATUS
Civil marriage           29775
Married                 196432
Separated                19770
Single / not married     45444
Unknown                      2
Widow                    16088
Name: NAME_FAMILY_STATUS, dtype: int64


Klien dengan status sudah menikah lebih banyak mengajukan pinjaman dibanding klien dengan status lainnya.

In [14]:
app_train_corr['TARGET'].sort_values()

EXT_SOURCE_2                   -0.160295
EXT_SOURCE_3                   -0.155892
EXT_SOURCE_1                   -0.098887
DAYS_EMPLOYED                  -0.044932
AMT_GOODS_PRICE                -0.039623
FLOORSMAX_AVG                  -0.039385
FLOORSMAX_MEDI                 -0.039157
FLOORSMAX_MODE                 -0.038377
REGION_POPULATION_RELATIVE     -0.037227
ELEVATORS_AVG                  -0.035853
ELEVATORS_MEDI                 -0.035552
ELEVATORS_MODE                 -0.034220
AMT_CREDIT                     -0.030369
TOTALAREA_MODE                 -0.030214
LIVINGAREA_AVG                 -0.029464
LIVINGAREA_MEDI                -0.029359
FLAG_DOCUMENT_6                -0.028602
LIVINGAREA_MODE                -0.027884
APARTMENTS_MEDI                -0.026841
APARTMENTS_AVG                 -0.026734
APARTMENTS_MODE                -0.025418
HOUR_APPR_PROCESS_START        -0.024166
FLAG_PHONE                     -0.023806
FLOORSMIN_AVG                  -0.021158
FLOORSMIN_MEDI  

Beberapa feature seperti DAYS_BIRTH, REGION_RATING_CLIENT_W_CITY, REGION_RATING_CLIENT, DAYS_LAST_PHONE_CHANGE, DAYS_ID_PUBLISH, REG_CITY_NOT_WORK_CITY memiliki korelasi tertinggi dengan TARGET diantara feature lainnya. Sementara itu korelasi terendah dengan TARGET ditunjukkan oleh feature EXT_SOURCE_2, EXT_SOURCE_3, EXT_SOURCE_1, dan DAYS_EMPLOYED. Beberapa feature tersebut selanjutnya akan dipilih untuk dilakukan pemodelan menggunakan algoritma machine learning yakni Logistic Regression dan Decision Tree. Selain feature-feature tersebut, feature bertipe string lainnya seperti CODE_GENDER, NAME_INCOME_TYPE, dan NAME_EDUCATION_TYPE juga dipilih untuk dilakukan pemodelan, namun terlebih dahulu dilakukan konversi dari tipe data string ke numerik.

In [15]:
# Memasukkan beberapa feature ke dalam dataset baru
app_train_2 = app_train[['TARGET','DAYS_BIRTH','REGION_RATING_CLIENT_W_CITY','REGION_RATING_CLIENT','DAYS_LAST_PHONE_CHANGE','DAYS_ID_PUBLISH','REG_CITY_NOT_WORK_CITY','DAYS_EMPLOYED','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','CODE_GENDER','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE']]
app_train_2.head()

,TARGET,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_LAST_PHONE_CHANGE,DAYS_ID_PUBLISH,REG_CITY_NOT_WORK_CITY,DAYS_EMPLOYED,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,CODE_GENDER,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE
0,1,-9461,2,2,-1134.0,-2120,0,-637,0.083037,0.262949,0.139376,M,Working,Secondary / secondary special
1,0,-16765,1,1,-828.0,-291,0,-1188,0.311267,0.622246,0.535276,F,State servant,Higher education
2,0,-19046,2,2,-815.0,-2531,0,-225,0.505998,0.555912,0.729567,M,Working,Secondary / secondary special
3,0,-19005,2,2,-617.0,-2437,0,-3039,0.505998,0.650442,0.535276,F,Working,Secondary / secondary special
4,0,-19932,2,2,-1106.0,-3458,1,-3038,0.505998,0.322738,0.535276,M,Working,Secondary / secondary special


In [16]:
# Melakukan konversi string ke numerik untuk data tingkat pendidikan
app_train_2['NAME_EDUCATION_TYPE'].unique()

new = {'Lower secondary':0, 'Secondary / secondary special':1, 'Incomplete higher':2, 'Higher education':3, 'Academic degree':4}

app_train_2['NAME_EDUCATION_TYPE'] = app_train_2['NAME_EDUCATION_TYPE'].replace(new)

In [17]:
# Melakukan konversi string ke numerik untuk data jenis kelamin dan sumber pendapatan
app_train_2['CODE_GENDER'].unique()

array(['M', 'F', 'XNA'], dtype=object)

In [18]:
# Melakukan konversi string ke numerik untuk data jenis kelamin dan sumber pendapatan
app_train_2['NAME_INCOME_TYPE'].unique()

array(['Working', 'State servant', 'Commercial associate', 'Pensioner',
       'Unemployed', 'Student', 'Businessman', 'Maternity leave'],
      dtype=object)

In [19]:
# Membuat dataset dummies/one-hot encoding untuk jenis kelamin dan sumber pendapatan
CODE_GENDER_dummies = pd.get_dummies(app_train_2['CODE_GENDER'], prefix='CODE_GENDER')
NAME_INCOME_TYPE_dummies = pd.get_dummies(app_train_2['NAME_INCOME_TYPE'], prefix='NAME_INCOME_TYPE')

# Menyatukan dataset dummies ke dalam dataset app_train_2
app_train_2 = pd.concat([app_train_2, CODE_GENDER_dummies, NAME_INCOME_TYPE_dummies], axis=1)

# Membuang feature yang memiliki tipe data kategori (sebelum dilakukan transformasi)
app_train_2 = app_train_2.drop(['CODE_GENDER', 'NAME_INCOME_TYPE'], axis=1)

In [20]:
app_train_2.head()

,TARGET,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_LAST_PHONE_CHANGE,DAYS_ID_PUBLISH,REG_CITY_NOT_WORK_CITY,DAYS_EMPLOYED,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,NAME_EDUCATION_TYPE,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working
0,1,-9461,2,2,-1134.0,-2120,0,-637,0.083037,0.262949,0.139376,1,0,1,0,0,0,0,0,0,0,0,1
1,0,-16765,1,1,-828.0,-291,0,-1188,0.311267,0.622246,0.535276,3,1,0,0,0,0,0,0,1,0,0,0
2,0,-19046,2,2,-815.0,-2531,0,-225,0.505998,0.555912,0.729567,1,0,1,0,0,0,0,0,0,0,0,1
3,0,-19005,2,2,-617.0,-2437,0,-3039,0.505998,0.650442,0.535276,1,1,0,0,0,0,0,0,0,0,0,1
4,0,-19932,2,2,-1106.0,-3458,1,-3038,0.505998,0.322738,0.535276,1,0,1,0,0,0,0,0,0,0,0,1


In [21]:
# Membagi dataset ke dalam x dan y
x = app_train_2.iloc[:, 1:22].values
y = app_train_2.iloc[:, 0].values

In [23]:
# Menangani data yang tidak seimbang (imbalanced data)
smote = SMOTE()

# Fit smote ke dalam x dan y
x_smote, y_smote = smote.fit_resample(x,y)

In [24]:
# Membagi data ke dalam dataset training dan testing
x_train, x_test, y_train, y_test = train_test_split(x_smote, y_smote, test_size=.2, random_state=0)

In [25]:
# Melakukan pemodelan machine learning dengan Logistic Regression
logreg = LogisticRegression()

# Fit logreg ke dalam training dataset
logreg = logreg.fit(x_train, y_train)
# Training model, melakukan prediksi
y_pred = logreg.predict(x_test)

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.



In [26]:
# Mengevaluasi Model Logistic Regression

# Akurasi
print('Akurasi Training:', logreg.score(x_train, y_train))
print('Akurasi Testing:', logreg.score(x_test, y_test))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:\n')
print(cm)

# Classification Report
cr = classification_report(y_test, y_pred)
print('Classification Report:\n')
print(cr)

Akurasi Training: 0.5523074440025028
Akurasi Testing: 0.5514835286314393
Confusion Matrix:

[[32035 24291]
 [26425 30324]]
Classification Report:

              precision    recall  f1-score   support

           0       0.55      0.57      0.56     56326
           1       0.56      0.53      0.54     56749

    accuracy                           0.55    113075
   macro avg       0.55      0.55      0.55    113075
weighted avg       0.55      0.55      0.55    113075



In [27]:
# Melakukan pemodelan machine learning dengan Decision Tree
desicion_tree = DecisionTreeClassifier()

# Fit logreg ke dalam training dataset
decision_tree = desicion_tree.fit(x_train, y_train)
# Training model, melakukan prediksi
y_pred = desicion_tree.predict(x_test)

In [28]:
# Mengevaluasi Model Desicion Tree

# Akurasi
print('Akurasi Training:', desicion_tree.score(x_train, y_train))
print('Akurasi Testing:', desicion_tree.score(x_test, y_test))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:\n')
print(cm)

# Classification Report
cr = classification_report(y_test, y_pred)
print('Classification Report:\n')
print(cr)

Akurasi Training: 1.0
Akurasi Testing: 0.8942737121379616
Confusion Matrix:

[[49826  6500]
 [ 5455 51294]]
Classification Report:

              precision    recall  f1-score   support

           0       0.90      0.88      0.89     56326
           1       0.89      0.90      0.90     56749

    accuracy                           0.89    113075
   macro avg       0.89      0.89      0.89    113075
weighted avg       0.89      0.89      0.89    113075

